In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import csv
import math
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
import pytorch_lightning as pl
import glob

## First, we slice the train images into 31 x 31 pixels with the ground truth in the middle

In [ ]:
def ndigit(n, x):
    x = str(x)
    while(len(x) < n):
        x = "0" + x
    return x

In [ ]:
def load_data(res, files = 20, test = False):
    j = 0
    if (test == True):
        path = "02"
    else:
        path = "train"
    res = int((res-1)/2)
    
    for n in range(files):
        image = np.load(f"images_{path}/images/image_{ndigit(3, n)}.npy")
        masks = np.load(f"masks_{path}/masks/mask_{ndigit(3, n)}.npy")
        masks = np.reshape(masks, (1024,1024,1))
        ground_truths_pos = np.array(np.where(masks != 0)).T

        # Add padding to every image edge in case there are ground truths which are too close to an edge
        padded_image = np.pad(image, ((0, 0), (res, res), (res, res)), mode='constant') 
        
        # Slice and save image
        for i in ground_truths_pos: 
            train_slice = (padded_image[:, i[0]-res : i[0]+res+1, i[1]-res : i[1]+res+1], np.array(masks[i[0], i[1], 0]))
            np.save(f"images_{path}/train/train_{ndigit(5, j)}.npy", train_slice)                                 
            j += 1

In [ ]:
#load_data(31)

## Then, we load the data and have a look

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(), # Converts an image to a Tensor
     transforms.ConvertImageDtype(torch.float),
     transforms.Lambda(lambda x : x / 3000),
     transforms.Lambda(lambda x : 1 if x > 1 else x), # clip images between 0 and 1
     transforms.Normalize((0.5)*12, # Mean for RGB
                          (0.5)*12) # Std for RGB
     ]) 

batch_size = 128

In [ ]:
directory = 'images_train/train'
file_paths = glob.glob(directory + '/*.npy')
trainset0 = [np.load(file_path, allow_pickle=True) for file_path in file_paths]
#delete every image that doesnt have the correct shape (!THIS MIGHT BE A REAL PROBLEM THAT NEEDS TO BE FIXED PROPERLY LATER!)
trainset = []
for pic in trainset0:
    if pic[0].shape == (10,31,31):
        trainset.append(pic)

In [ ]:
def enrich_channels(trainset, veggie, moisture):
# structure of the data: 
#trainset[pic_no][0][CHANNEL][Horizontal][VERTIKAL] -> Intensity 
#trainset[pic_no][1]-> Ground truth 
    print("Shape vorher: Liste mit 10,31,31 Bildern")
    counter = 0
    trainset = trainset

    if veggie:
        pic_no = 0
        for pic in trainset:
            counter += 1
            pixel_values = pic[0][:][:]
            channel8 = pixel_values[6]
            channel4 = pixel_values[2]
            channels = pic[0].shape[0]
            #print("Chanels:", channels)
            width = pic[0].shape[1]
            height = pic[0].shape[2]
            vegetation_array = np.divide((np.subtract(channel8, channel4)), np.add(channel8, channel4))
            trainset_transformed = np.append(trainset[pic_no][0], vegetation_array)
            trainset1_transformed = np.reshape(trainset_transformed, (channels + 1, width, height))
            trainset[pic_no] = (trainset1_transformed, trainset[pic_no][1])
            pic_no += 1

        print("Added Vegetation (B8-B4)/(B8+B4)")

    if moisture:
        pic_no = 0
        for pic in trainset:
            pixel_values = pic[0][:][:]
            channel8a = pixel_values[7]
            channel11 = pixel_values[8]
            channels = pic[0].shape[0]
            #print("Chanels:", channels)

            width = pic[0].shape[1]
            height = pic[0].shape[2]
            moisture_array = np.divide((np.subtract(channel8a, channel11)), np.add(channel8a, channel11))
            trainset_transformed = np.append(trainset[pic_no][0], moisture_array)
            trainset1_transformed = np.reshape(trainset_transformed, (channels + 1, width, height))
            print(trainset1_transformed.shape)
            trainset[pic_no] = (trainset1_transformed, trainset[pic_no][1]) # append ground truth in tupel 
            pic_no += 1

        print("Added Moisture (B8A-B11)/(B8A+B11)")

    return trainset



In [ ]:
class MyDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __getitem__(self, index):
        item = self.data[index]

        # Extract the image from the tuple
        image = item[0]

        # Apply transformations if specified
        if self.transform is not None:
            image = self.transform(image)

        # Return the transformed image and the remaining items in the tuple
        return image, *item[1:]

    def __len__(self):
        return len(self.data)


In [ ]:
trainset = enrich_channels(trainset, True, True)
#trainset[pic_no][0][channel][h][w] -> pixel value
#trainset[pic_no][1] -> Ground truth 

In [ ]:
# Calculate the sizes of the training set and validation set
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size

# Split trainset into trainset and valset
trainset, valset = random_split(trainset, [train_size, val_size])

# Create data loaders for the training set and validation set
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0)
validloader = DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=0)


In [ ]:
f, axarr = plt.subplots(1,10, figsize=(12, 12))
for i in range(10):
    X,y = trainset[i]
    X,y = X.transpose(0,-1).transpose(0,1) * 0.5 + 0.5, y
    axarr[i].imshow(X)
    axarr[i].axis('off')
    axarr[i].set_title(f'{y}', fontsize='small')

## Next, we define the model and train it

In [ ]:
class MyCNNModel(pl.LightningModule): # New! def init(self, layers, lr=0.01, classes=None): super().init() # <- Very important! self.lr = lr self.classes = classes ## Build model self.layers = nn.Sequential(layers) # Create a sequential model

    def __init__(self, *layers, classes=None):
        super().__init__()

        self.lr = 0.01  # Assign the learning rate here
        self.classes = classes

        self.layers = nn.Sequential(*layers)  # Create a sequential model
        
    def forward(self, X):
        return self.layers(X)

    def predict(self, X):
        with torch.no_grad():
            y_hat = self(X).argmax(1)
        if self.classes is not None:
            y_hat = [self.classes[i] for i in y_hat]
        return y_hat

    def training_step(self, batch, batch_idx, log_prefix='train'): # New !
        X, y = batch # Tuple with (X,y) in our case
        y_hat = self(X)
        loss = nn.MSELoss(y_hat, y)
        self.log(f"{log_prefix}_loss", loss.item(), on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx): # New!
        with torch.no_grad():
            return self.training_step(batch, batch_idx, log_prefix='valid')

    def configure_optimizers(self):
        # Adam with Weight Decay (Most commonly used)
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=0.01)

        # Simplest scheduler is ReduceLROnPlateau. This scheduler reduces the learning rate by 0.1
        # if the val_loss has not decreased within the last 10 epochs.
        scheduler = {
            # REQUIRED: The scheduler instance
            "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=10, verbose=True),
            # The unit of the scheduler's step size, could also be 'step'.
            # 'epoch' updates the scheduler on epoch end whereas 'step'
            # updates it after a optimizer update.
            "interval": "epoch",
            # How many epochs/steps should pass between calls to
            # `scheduler.step()`. 1 corresponds to updating the learning
            # rate after every epoch/step.
            "frequency": 1,
            # Metric to to monitor for schedulers like `ReduceLROnPlateau`
            "monitor": "val_loss",
            # If set to `True`, will enforce that the value specified 'monitor'
            # is available when the scheduler is updated, thus stopping
            # training if not found. If set to `False`, it will only produce a warning
            "strict": True,
            # If using the `LearningRateMonitor` callback to monitor the
            # learning rate progress, this keyword can be used to specify
            # a custom logged name
            "name": None,
        }
        return {"optimizer": optimizer, 'lr-scheduler': scheduler}

## Implement model

In [ ]:
# Implements entry to SepConv2d, see Lang et al. (2019), p. 6
class MyEntryLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.proj_out = nn.Conv2d(in_channels, out_channels[len(out_channels)-1], (1,1))

        self.entry_blocks = nn.ModuleList()
        for i in range(len(out_channels)):
            self.entry_blocks.append(nn.Sequential(
                nn.Conv2d(in_channels, out_channels[i], (1, 1)),
                nn.BatchNorm2d(out_channels[i]),
                nn.ReLU()
            ))
            in_channels = out_channels[i]  # Update in_channels for next iteration

    def forward(self, x):
        x_entry = x
        for i in range(len(self.out_channels)):
            x_entry = self.entry_blocks[i](x_entry)
        x = self.proj_out(x)
        return (x + x_entry)

In [ ]:
# Implements SepConv2D
class MySepConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel, **kwargs):
        super().__init__()
        if in_channels == out_channels:
            self.proj_out = nn.Identity()
        else:
            self.proj_out = nn.Conv2d(in_channels, out_channels, (1,1), **kwargs)

        self.sep_conv_block = nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(in_channels, in_channels, kernel, groups=in_channels, **kwargs), # depthwise SepConv
            nn.Conv2d(in_channels, out_channels, (1,1), **kwargs), # pointwise SepConv
            nn.BatchNorm2d(out_channels)
        )
    
    def forward(self, x):
        x_sep_conv = self.sep_conv_block(x)
        x_sep_conv_2 = self.sep_conv_block(x_sep_conv) # performs second SepConv, see Lang et al. (2019), p. 6
        x = self.proj_out(x)
        return (x + x_sep_conv_2) # adds original input and sep_conv_2 output

In [ ]:
tree_model = MyCNNModel(
    MyEntryLayer(10, [128, 256, 512]), # increase number of channels to 512
    MySepConvLayer(512, 512, (3,3), padding='same'),
    MySepConvLayer(512, 512, (3,3), padding='same'),
    MySepConvLayer(512, 512, (3,3), padding='same'),
    MySepConvLayer(512, 512, (3,3), padding='same'),
    MySepConvLayer(512, 512, (3,3), padding='same'),
    MySepConvLayer(512, 512, (3,3), padding='same'),
    nn.AdaptiveMaxPool2d(1),
    nn.Flatten(1),
    nn.Linear(512, 1)
)

In [ ]:
# New, we need a trainer class
from pytorch_lightning.callbacks import RichProgressBar, RichModelSummary
trainer1 = pl.Trainer(devices=1, accelerator="cpu", precision='32', max_epochs=1,
                      callbacks=[RichProgressBar(refresh_rate=50),
                                 RichModelSummary(3),
                                ])

In [ ]:
print(list(tree_model.parameters()))


In [ ]:
trainer1.fit(tree_model, trainloader, validloader)

## Now, we can apply it

In [ ]:
f, axarr = plt.subplots(1,10, figsize=(12, 12))
for i in range(10):
    X,y = testset[i]
    y_hat = tree_model.predict(X.unsqueeze(0))[0]
    X,y = X.transpose(0,-1).transpose(0,1) * 0.5 + 0.5, testset.classes[y]
    axarr[i].imshow(X)
    axarr[i].axis('off')
    axarr[i].set_title(f'{y} - {y_hat}', fontsize='small')